In [10]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True,
)

groq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

In [8]:
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(override=True)
qw_embedding = DashScopeEmbeddings(
    model="text-embedding-v2", dashscope_api_key=os.getenv('DASHSCOPE_API_KEY')
)

In [3]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["qwen-plus"],
    },
)
#
# Sample content of OAI_CONFIG_LIST file below:
#
# [
#   {
#     "model": "gpt-4",
#     "api_key": "your openai api key"
#   }
# ]
#

In [4]:
gpt4_api_key = config_list[0]["api_key"]

In [5]:
gpt4_api_key

'sk-01c5003340c3453b934052d737d45e01'

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
# from langchain.chains import ConversationalRetrievalChain

In [7]:
loaders = [PyPDFLoader('./uniswap_v3.pdf')]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [9]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=qw_embedding
)
vectorstore.add_documents(docs)

['31f3642b-9dbc-4f33-8a46-2c3d2ac4b039',
 '01ead5d0-5179-4680-9521-73405d1e77d2',
 '57210b9c-0374-4766-9da7-e7b12a20cab5',
 '62e7e29f-c953-4838-ac6e-4c5c1752bfcf',
 '02c80417-ff10-43b3-a469-fbc46b5977d4',
 '887bcc34-6535-4983-a605-071df82049f4',
 '144a981e-9fb0-4a49-8c17-3045c8f0ce70',
 'bcfd17de-8fe1-4da5-95b8-4a0d20bd6080',
 '4712fdac-159d-4e8e-855e-31d84d1a34bf',
 '6458415d-5405-47df-b099-979ad6d0c26f',
 'd5a551e6-7f3d-41c1-be23-992410bb0121',
 '89ae7476-d3c7-4c52-84e4-dbbca72b07fb',
 '1fc7fe89-79a5-462d-a44b-6344fbcfab90',
 '88e479ab-4e94-4803-8618-8364f7704ccf',
 'b48b3b4f-4237-457b-8dbb-d03f75734f1b',
 '3c80d683-bfc2-4c58-9686-1b6a10aa2218',
 'c5e1d8cc-b74c-45db-8c2a-d103ee65986c',
 'a43cc7ec-c3af-4b61-bb71-1ffdccb8629f',
 '914e5fcb-10f6-45c9-9863-eb71ba8c8cae',
 'af068d74-cc81-4c8c-8c66-710f2f292bd4',
 'aa46289b-0422-4e5a-a120-2a40cc7149ea',
 'e13fa78c-84ce-4ffa-a686-8e6a07df9ccd',
 '1bd8bd2f-fb2e-4439-8493-aab8a967b81a',
 'ea071099-6dbe-4e60-9ef2-53d14105687f',
 '2c2b5d3a-2e15-

In [11]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    qw_llm_openai,
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [12]:
result = qa.invoke(({"question": "What is uniswap?"}))

In [13]:
result

{'question': 'What is uniswap?',
 'chat_history': [HumanMessage(content='What is uniswap?'),
  AIMessage(content='Uniswap is a non-custodial automated market maker (AMM) implemented for the Ethereum Virtual Machine (EVM). It was developed by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, Dan Robinson, and others. The core version of Uniswap v3 was released in March 2021.')],
 'answer': 'Uniswap is a non-custodial automated market maker (AMM) implemented for the Ethereum Virtual Machine (EVM). It was developed by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, Dan Robinson, and others. The core version of Uniswap v3 was released in March 2021.'}

In [14]:
def answer_uniswap_question(question):
    response = qa.invoke({"question": question})
    return response["answer"]

4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [15]:
llm_config = {
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [17]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": ".", "use_docker": False},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_uniswap_question": answer_uniswap_question}
)

In [18]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.


--------------------------------------------------------------------------------


TypeError: Completions.create() got an unexpected keyword argument 'request_timeout'